In [343]:
import numpy as np
import math
from sklearn import datasets
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

In [302]:
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state = 1)

In [347]:
current_entropy=entropy(x_train,y_train,len(x_train))

CustomDecisionTreeClassifier(x_train,y_train,iris.feature_names,current_entropy,0,0,iris.feature_names[0])

pure node
y_train [0]
current entropy is 0.0
1
petal width (cm) 0.202437982551221

pure node
y_train [0 0 0]
current entropy is 0.0
3
petal width (cm) 0.09769178693160412

pure node
y_train [0 0 0 0 0]
current entropy is 0.0
5
petal width (cm) 0.036777106805558527

pure node
y_train [0 0 0 0 0]
current entropy is 0.0
5
petal width (cm) 0.3171042163605767

here
y_train [0 0 2 0 1 0]
current entropy is 1.2516291673878228
6
petal width (cm) 0

here
y_train [0 1 0 0 0 1 0 0]
current entropy is 0.8112781244591328
8
petal width (cm) 0

pure node
y_train [0 0 0 0 0 0]
current entropy is 0.0
6
petal width (cm) 0.2339774775966272

here
y_train [1 0 0]
current entropy is 0.9182958340544896
3
petal width (cm) 0

pure node
y_train [0 0]
current entropy is 0.0
2
petal width (cm) 0.2919693065629077

here
y_train [0 1 1 1]
current entropy is 0.8112781244591328
4
petal width (cm) 0

here
y_train [1 2 1]
current entropy is 0.9182958340544896
3
petal width (cm) 0

here
y_train [0 1 2 1 0 1 1]
current en

In [330]:
def entropy(x_train,y_train,totalDataPoints):
    
    count=np.unique(y_train,return_counts=True)
    totalPoints= len(x_train)
    
    ent=0
    
    for i in range(len(count[0])):
        ent=ent+-1*((count[1][i]/totalPoints)*(math.log(count[1][i]/totalPoints,2)))

    return ent
    
    
        
    

In [335]:
def calculate_gain_ratio(x_train,y_train,selected_feature,previous_entropy):
    
    x=x_train[:,selected_feature]
    x=np.sort(x)
    
    totalDataPoints=len(x)
    
    splitPoint=x[0]
    
    ele1=x[0]
    
    max_gain_ratio=0
    gain_ratio=0
    max_total_entropy=0
    
    for i in range(1,len(x)):
        splitInfo=0
        ele2=x[i]
        mid=(ele1+ele2)/2
        ele1=ele2
        
        x1=x[x<=mid]
        
        e1=entropy(x1,y_train[x<=mid],len(x1))
        
        if(len(x1)!=0):
            splitInfo=splitInfo+ (-1)*((len(x1)/len(x))*(math.log(len(x1)/len(x),2)))
        
        x2=x[x>mid]
        
        e2=entropy(x2,y_train[x>mid],len(x2))
        
        if(len(x2)!=0):
            splitInfo=splitInfo+ (-1)*((len(x2)/len(x))*(math.log(len(x2)/len(x),2)))
        
        totalEntropy = (len(x1)/totalDataPoints)*e1 + (len(x2)/totalDataPoints)*e2
        
        info_gain = abs(totalEntropy-previous_entropy)
        
        if splitInfo!=0:
            gain_ratio = info_gain/splitInfo
        
        if(gain_ratio>max_gain_ratio):
            max_gain_ratio=gain_ratio
            splitPoint=mid
            max_total_entropy=totalEntropy
            
#         print(totalEntropy)
        
        
    
    return max_gain_ratio,splitPoint,max_total_entropy

In [346]:
def CustomDecisionTreeClassifier(x_train,y_train,features,previous_entropy,selected_feature,previous_gain_ratio,f_name):
   
    if(len(set(y_train))==0):
        return
    
    if len(set(y_train))==1:
        print('pure node')
        print('y_train',y_train)
        print('current entropy is',entropy(x_train,y_train,len(x_train)))
        print(len(x_train))
        print(features[selected_feature],previous_gain_ratio)
        print()
        return
    
    if len(features)==0:
        print('here')
        print('y_train',y_train)
        print('current entropy is',entropy(x_train,y_train,len(x_train)))
        print(len(x_train))
        print(f_name,previous_gain_ratio)
        print()
        return
    
   
        
    max_gain_ratio=0
    selected_split_point=0
    selected_feature=0
    ent=0
    for f in range(len(features)):
        
        gain_ratio_and_split_point = calculate_gain_ratio(x_train,y_train,f,previous_entropy)
        gain_ratio=gain_ratio_and_split_point[0]
        split_point=gain_ratio_and_split_point[1]
#         print('split_point',split_point)
        if max_gain_ratio<gain_ratio:
            max_gain_ratio=gain_ratio
            selected_feature=f
            selected_split_point=split_point
            ent=gain_ratio_and_split_point[2]
            
    
    x1=len(x_train[x_train[:,selected_feature]<=split_point])
    x2=len(x_train[x_train[:,selected_feature]>split_point])
    f_name=features[selected_feature]
    if(x1==0 or x2==0):
        
        if x1!=0:
            CustomDecisionTreeClassifier(x_train[x_train[:,selected_feature]<=split_point],y_train[x_train[:,selected_feature]<=split_point],np.delete(features,selected_feature),ent,selected_feature,max_gain_ratio,f_name)
        if x2!=0:
            CustomDecisionTreeClassifier(x_train[x_train[:,selected_feature]>split_point],y_train[x_train[:,selected_feature]>split_point],np.delete(features,selected_feature),ent,selected_feature,max_gain_ratio,f_name)
    else:
        CustomDecisionTreeClassifier(x_train[x_train[:,selected_feature]<=split_point],y_train[x_train[:,selected_feature]<=split_point],features,ent,selected_feature,max_gain_ratio,f_name)
        CustomDecisionTreeClassifier(x_train[x_train[:,selected_feature]>split_point],y_train[x_train[:,selected_feature]>split_point],features,ent,selected_feature,max_gain_ratio,f_name)
            
            
            